In [1]:
import pandas as pd
import pymysql
from tqdm import tqdm

## Find the Synonym Of The Medication Class

In [2]:
Medication = pd.read_csv("type 2 diabetes medication class.csv")

In [3]:
Medication = Medication.drop_duplicates(subset=['MedicationCui', 'ClassCui'])

In [4]:
database = pymysql.connect(host="localhost",user= "root", password="Zhangsinian976", database="umls")
cursor = database.cursor()

In [5]:
command = '''select distinct cui2,str
from mrrel,mrconso
where cui1 = %s and cui2 != %s and cui = cui2 and (rel = "SY" or rela = "mapped_to");'''

In [6]:
Data = pd.DataFrame(columns=["MedicationCui","MedicationStr","ClassCui","ClassStr","ClassCui(Synonym)","ClassStr(Synonym)"])

for index,row in tqdm(Medication.iterrows()):
    cursor.execute(command,(row["ClassCui"],row["ClassCui"]))
    result = cursor.fetchall()
    if(result == ()):
        Data = pd.concat([Data,row.to_frame().T],ignore_index=True)
        continue
    Temp = pd.DataFrame(result,columns=["ClassCui(Synonym)","ClassStr(Synonym)"])
    duplicated_row = pd.concat([row.to_frame().T] * len(Temp), ignore_index=True)
    combined = pd.concat([duplicated_row.reset_index(drop=True), Temp.reset_index(drop=True)], axis=1)
    Data = pd.concat([Data,combined],ignore_index=True)

0it [00:00, ?it/s]

324it [3:28:45, 38.66s/it]


In [7]:
cursor.close()
database.close()

In [8]:
Medication = Data.drop_duplicates(subset=['MedicationCui','ClassCui','ClassCui(Synonym)'])

In [19]:
Data.drop_duplicates(subset=['MedicationCui','ClassCui','ClassCui(Synonym)']).to_csv("SynonymClass(T2D).csv",index=False,encoding='utf-8-sig')

## Filter Medication Class

In [9]:
MedicationClass = pd.read_csv('T2DMedicationClass.txt', sep='|', skipinitialspace=True)

In [10]:
key_cui = MedicationClass['Key Cui '].str.replace(r'\[|\]', '',regex=True).str.strip().str.split(', ').explode()
superior_key_cui = MedicationClass['Superior Key Cui '].str.replace(r'\[|\]', '',regex=True).str.strip().str.split(', ').explode()
all_cui = pd.concat([key_cui, superior_key_cui]).drop_duplicates().tolist()

In [11]:
Medication = Medication.loc[(Medication['ClassCui'].str.strip().isin(all_cui) | Medication['ClassCui(Synonym)'].str.strip().isin(all_cui))]

In [12]:
Medication

,MedicationCui,MedicationStr,ClassCui,ClassStr,ClassCui(Synonym),ClassStr(Synonym)
6,C0000992,acetohexamide,C0020616,Hypoglycemic Agents,C0017687,glukagon
61,C0000992,acetohexamide,C0038766,Sulfonylurea Compounds,C0061323,glimepiride
79,C0000992,acetohexamide,C0038766,Sulfonylurea Compounds,C0061328,gliquidone
98,C0000992,acetohexamide,C0038766,Sulfonylurea Compounds,C0061329,glisolamide
103,C0000992,acetohexamide,C0038766,Sulfonylurea Compounds,C0061330,glisoxepide
...,...,...,...,...,...,...
17574,C3491971,insulin degludec,C0021641,Insulin,C4549738,neutral protamine lispro insulin
17576,C3491971,insulin degludec,C0021641,Insulin,C4742625,MK-2640
17579,C3555622,dapagliflozin propanediol,C2353951,dapagliflozin,NaN,NaN
17580,C3555622,dapagliflozin propanediol,C3273807,Sodium-Glucose Transporter 2 Inhibitors,NaN,NaN


## Filter Rxnorm Medication

In [13]:
database = pymysql.connect(host="localhost",user= "root", password="Zhangsinian976", database="umls")
cursor = database.cursor()

In [14]:
command = '''select * from rxnconso where str = %s'''

In [15]:
for index,row in tqdm(Medication.iterrows()):
    cursor.execute(command,row["MedicationStr"])
    result = cursor.fetchall()
    if len(result) == 0:
        Medication.drop(index, inplace=True)

2416it [13:38,  2.95it/s]


In [16]:
cursor.close()
database.close()

In [17]:
Medication.sort_index().to_csv('T2D medication class(filtered).csv',encoding="utf_8_sig",index=False)

In [18]:
Medication["MedicationCui"].unique().shape

(43,)